In [1]:
import pickle, random, copy, platform, os
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [2]:
def vessel_img():
    with open('vessel', 'rb') as vessel:
        data = pickle.load(vessel)
        band1 = data[0]
        band2 = data[1]

    return band1, band2

In [3]:
#band1
band1 = vessel_img()[0]
band2 = vessel_img()[1]

In [4]:
X1 = band1.reshape(-1, 5625)
X2 = band2.reshape(-1, 5625)

In [5]:
'''X1 = np.array([X1[0] for i in range(len(X1))])
X2 = np.array([X2[0] for i in range(len(X1))])'''

'X1 = np.array([X1[0] for i in range(len(X1))])\nX2 = np.array([X2[0] for i in range(len(X1))])'

In [6]:
# GAN Modeling

import matplotlib.pyplot as plt

total_epoch = 10000
batch_size = 37 #851=37*23
learning_rate = 0.00002
# layer option
n_noise = 256 # the size of noise 
n_hidden = 512
n_hidden2 = 1024
n_hidden3 = 2024
n_input = 75*75

In [7]:
# X is the input, we do not use the labels Y of images
X = tf.placeholder(tf.float32, [None, n_input])
# Z is the noise 
Z = tf.placeholder(tf.float32, [None, n_noise])

In [8]:
g_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], mean=0, stddev=0.01, dtype=tf.float32))
g_b1 = tf.Variable(tf.random_normal([n_hidden],stddev=0.01))

g_W2 = tf.Variable(tf.random_normal([n_hidden, n_hidden2], mean=0, stddev=0.01, dtype=tf.float32))
g_b2 = tf.Variable(tf.random_normal([n_hidden2],stddev=0.01))
g_W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3], mean=0, stddev=0.01, dtype=tf.float32))
g_b3 = tf.Variable(tf.random_normal([n_hidden3],stddev=0.01))
                   
g_W4 = tf.Variable(tf.random_normal([n_hidden3, n_input], mean=0, stddev=0.01, dtype=tf.float32))
g_b4 = tf.Variable(tf.random_normal([n_input],stddev=0.01))

In [9]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, g_W1) + g_b1)
    hidden2 = tf.nn.relu(tf.matmul(hidden, g_W2) + g_b2)
    hidden3 = tf.nn.relu(tf.matmul(hidden2, g_W3) + g_b3)
    output = tf.nn.sigmoid(tf.matmul(hidden3, g_W4) + g_b4)
    return output

In [10]:
d_W1 = tf.Variable(tf.random_normal([n_input, n_hidden3], mean=0, stddev=0.01, dtype=tf.float32))
d_b1 = tf.Variable(tf.random_normal([n_hidden3],stddev=0.01))

d_W2 = tf.Variable(tf.random_normal([n_hidden3, n_hidden2], mean=0, stddev=0.01, dtype=tf.float32))
d_b2 = tf.Variable(tf.random_normal([n_hidden2],stddev=0.01))
d_W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden], mean=0, stddev=0.01, dtype=tf.float32))
d_b3 = tf.Variable(tf.random_normal([n_hidden],stddev=0.01))

d_W4 = tf.Variable(tf.random_normal([n_hidden, 1], mean=0, stddev=0.01, dtype=tf.float32))
d_b4 = tf.Variable(tf.random_normal([1],stddev=0.01))

In [11]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, d_W1) + d_b1)
    hidden2 = tf.nn.relu(tf.matmul(hidden, d_W2) + d_b2)
    hidden3 = tf.nn.relu(tf.matmul(hidden2, d_W3) + d_b3)
    output = tf.nn.sigmoid(tf.matmul(hidden3, d_W4) + d_b4)
    return output

In [12]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [13]:
# Generate the random image
G = generator(Z)
# Discriminate the generated image
D_generate = discriminator(G)
# Discriminate the real image
D_real = discriminator(X)

In [14]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))

In [15]:
loss_D_generate = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_generate, labels=tf.zeros_like(D_generate)))

In [16]:
loss_D = loss_D_real + loss_D_generate

In [17]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_generate, labels=tf.ones_like(D_generate)))

In [18]:
d_var_list = [d_W1, d_b1, d_W2, d_b2, d_W3, d_b3, d_W4, d_b4]
g_var_list = [g_W1, g_b1, g_W2, g_b2, g_W3, g_b3, g_W4, g_b4]

In [19]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D,var_list=d_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,var_list=g_var_list)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(851/batch_size)
loss_val_D, loss_val_G = 0, 0

In [21]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs= X1[i*batch_size:(i+1)*batch_size]
        noise = get_noise(batch_size, n_noise)

        # learn the discriminator and generator.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    #########
    # save the images which is generated
    ######
    if epoch == 0 or (epoch + 1) % 5 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (75, 75)), cmap=plt.cm.bone, interpolation='nearest')

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
        plt.close(fig)
        print(samples[0])
print('최적화 완료!')

Epoch: 0000 D loss: 1.23 G loss: 0.5137
[ 0.49949601  0.49517575  0.49776331 ...,  0.4997144   0.49991229
  0.50121123]
Epoch: 0001 D loss: 1.071 G loss: 0.6379
Epoch: 0002 D loss: 1.011 G loss: 0.6886
Epoch: 0003 D loss: 1.008 G loss: 0.6917
Epoch: 0004 D loss: 1.007 G loss: 0.6921
[ 0.48761106  0.47097969  0.49397627 ...,  0.48557425  0.46923754
  0.47589582]
Epoch: 0005 D loss: 1.007 G loss: 0.6921
Epoch: 0006 D loss: 1.01 G loss: 0.69
Epoch: 0007 D loss: 1.075 G loss: 0.6327
Epoch: 0008 D loss: 1.253 G loss: 0.4963
Epoch: 0009 D loss: 1.271 G loss: 0.4844
[ 0.0003545   0.00015297  0.00057447 ...,  0.00031647  0.00015614
  0.00018815]
Epoch: 0010 D loss: 1.27 G loss: 0.4851
Epoch: 0011 D loss: 1.264 G loss: 0.4887
Epoch: 0012 D loss: 1.255 G loss: 0.4942
Epoch: 0013 D loss: 1.244 G loss: 0.5018
Epoch: 0014 D loss: 1.228 G loss: 0.5125
[  7.61970223e-05   3.81669306e-05   1.39106225e-04 ...,   6.73460818e-05
   4.11425026e-05   5.23954768e-05]
Epoch: 0015 D loss: 1.207 G loss: 0.527


KeyboardInterrupt: 